In [1]:
import matplotlib as mpl
import os
mpl.rcParams["figure.facecolor"] = "white"

mpl.rcParams["axes.facecolor"] = "white"
mpl.rcParams["savefig.facecolor"] = "white"
%matplotlib inline

In [2]:

import pandas as pd
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from calibration_dataset import Tell1Dataset

class MyDS(Tell1Dataset):
	filename_format = '%Y-%m-%d'
	filename_regex_format = r'\d{4}-\d{2}-\d{2}.csv'

#datapath =
datapath = "data/calibrations/"
#datapath = "..\..\data\calibrations"
data_list = MyDS.get_filepaths_from_dir(datapath)
mds = MyDS(data_list, read=True)


100%|█████████████████████████████████████████| 30/30 [00:06<00:00,  4.89it/s]


In [4]:
%load_ext tensorboard

In [5]:
from networks import VeloDecoderSmall, VeloEncoderSmall, VeloAutoencoderLt
import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader, TensorDataset

In [6]:
mds.dfh['R'].df

,type,sensor,datetime,Zmod,slot_label,mod_nr,mod_type,sensor_type,sensor_number,channel0,...,channel2038,channel2039,channel2040,channel2041,channel2042,channel2043,channel2044,channel2045,channel2046,channel2047
2,hit_threshold,0.0,2010-08-16,-175,VL01L,mod_0,VELO_phi,R,#0,17.0,...,13.0,12.0,12.0,13.0,12.0,13.0,12.0,12.0,12.0,13.0
5,hit_threshold,0.0,2010-08-18,-175,VL01L,mod_0,VELO_phi,R,#0,17.0,...,12.0,12.0,12.0,12.0,13.0,13.0,12.0,13.0,12.0,12.0
8,hit_threshold,0.0,2011-03-07,-175,VL01L,mod_0,VELO_phi,R,#0,18.0,...,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0,13.0
11,hit_threshold,0.0,2011-05-01,-175,VL01L,mod_0,VELO_phi,R,#0,16.0,...,13.0,14.0,14.0,13.0,13.0,13.0,13.0,13.0,14.0,13.0
14,hit_threshold,0.0,2011-06-18,-175,VL01L,mod_0,VELO_phi,R,#0,10.0,...,13.0,13.0,13.0,13.0,13.0,13.0,14.0,14.0,13.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3767,hit_threshold,41.0,2016-11-07,750,VL25R,mod_41,VELO_R,R,#41,17.0,...,13.0,14.0,13.0,14.0,14.0,13.0,14.0,14.0,13.0,14.0
3770,hit_threshold,41.0,2016-11-11,750,VL25R,mod_41,VELO_R,R,#41,17.0,...,13.0,13.0,13.0,14.0,14.0,13.0,14.0,14.0,14.0,14.0
3773,hit_threshold,41.0,2017-05-21,750,VL25R,mod_41,VELO_R,R,#41,17.0,...,13.0,13.0,13.0,13.0,13.0,13.0,14.0,13.0,13.0,14.0
3776,hit_threshold,41.0,2017-06-21,750,VL25R,mod_41,VELO_R,R,#41,17.0,...,13.0,14.0,13.0,14.0,13.0,13.0,14.0,14.0,13.0,13.0


In [7]:
dfh = mds.dfh.df.iloc[:,9:]
dfh_r = mds.dfh['R'].df.iloc[:,9:]
dfh_phi = mds.dfh['phi'].df.iloc[:,9:]

In [8]:
dfp = mds.dfp.df.iloc[:,9:]
dfp_r = mds.dfp['R'].df.iloc[:,9:]
dfp_phi = mds.dfp['phi'].df.iloc[:,9:]

In [9]:
from sklearn.model_selection import train_test_split
from pytorch_lightning.loggers import TensorBoardLogger
import os



In [10]:
def make_loader(dataset):
    train, test = train_test_split(dataset, test_size=0.2)
    train_target = torch.tensor(train.values, dtype=torch.float)
    train_data = torch.tensor(train.values, dtype=torch.float)
    test_target = torch.tensor(test.values, dtype=torch.float)
    test_data = torch.tensor(test.values, dtype=torch.float)
    train_tensor = TensorDataset(train_data, train_target) 
    test_tensor = TensorDataset(test_data, test_target) 
    train_loader = DataLoader(dataset = train_tensor)
    test_loader = DataLoader(dataset = test_tensor)
    return train_loader, test_loader

In [11]:
from datetime import datetime
datetime.now().strftime("%d_%m_%Y_%H_%M_%S")

'15_12_2020_23_04_38'

In [12]:
def make_model_trainer(s, version):
    s = 2048
    dec = VeloDecoderSmall(s)
    enc = VeloEncoderSmall(s)
    model = VeloAutoencoderLt(enc, dec)

    logger = TensorBoardLogger(
        save_dir=os.getcwd(),
        version=version+datetime.now().strftime("_%d_%m_%Y_%H_%M_%S"),
        name='lightning_logs'
    )
    tr = pl.Trainer(logger=logger, max_epochs=50, gpus=1)
    return model, tr



In [13]:
def run_experiment(dataset, version):
    train_loader, test_loader = make_loader(dataset)
    s = dataset.shape[1]
    model, tr = make_model_trainer(s, version)
    tr.fit(model, train_loader, test_loader)

In [14]:
run_experiment(dfh, "dfh_small")


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type             | Params
------------------------------------------
0 | enc  | VeloEncoderSmall | 82.4 K
1 | dec  | VeloDecoderSmall | 84.4 K
------------------------------------------
166 K     Trainable params
0         Non-trainable params
166 K     Total params


Epoch 6:  80%|████ | 2016/2520 [00:40<00:10, 49.58it/s, loss=48.4, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 8:  80%|████ | 2016/2520 [00:23<00:05, 86.06it/s, loss=43.4, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 9:  80%|████ | 2016/2520 [00:23<00:05, 86.07it/s, loss=38.1, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 10:  92%|███▋| 2320/2520 [00:24<00:02, 96.25it/s, loss=37.2, v_num=4_38]


Epoch 11:  80%|███▏| 2016/2520 [00:21<00:05, 95.04it/s, loss=35.6, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 13:  80%|███▏| 2016/2520 [00:21<00:05, 92.20it/s, loss=33.7, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 14:  80%|███▏| 2016/2520 [00:24<00:06, 81.34it/s, loss=33.1, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 15:  80%|███▏| 2016/2520 [00:28<00:07, 71.45it/s, loss=32.4, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 16:  80%|███▏| 2016/2520 [00:26<00:06, 77.10it/s, loss=31.6, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 17:  80%|████▊ | 2016/2520 [00:26<00:06, 76.76it/s, loss=31, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 18:  80%|███▏| 2016/2520 [00:27<00:06, 73.14it/s, loss=30.5, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 19:  80%|███▏| 2016/2520 [00:27<00:06, 72.29it/s, loss=30.3, v_num=4_38]


Validating: 0it [00:00, ?it/s]
Epoch 20:  80%|███▏| 2016/2520 [00:25<00:06, 79.44it/s, loss=30.3, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 21:  80%|███▏| 2016/2520 [00:25<00:06, 79.08it/s, loss=30.3, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 22:  80%|███▏| 2016/2520 [00:25<00:06, 78.15it/s, loss=29.6, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 23:  80%|███▏| 2016/2520 [00:25<00:06, 78.32it/s, loss=29.4, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 24:  80%|███▏| 2016/2520 [00:25<00:06, 78.85it/s, loss=28.1, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 25:  80%|████▊ | 2016/2520 [00:25<00:06, 79.04it/s, loss=27, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 26:  80%|████▊ | 2016/2520 [00:25<00:06, 79.21it/s, loss=26, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 27:  80%|███▏| 2016/2520 [00:25<00:06, 79.23it/s, loss=25.5, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 27:  95%|███▊| 2400/2520 [00:26<00:01, 91.54it/s, loss=25.5, v_num=4_38]


Epoch 28:  80%|███▏| 2016/2520 [00:26<00:06, 75.70it/s, loss=25.2, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 29:  80%|███▏| 2016/2520 [00:29<00:07, 68.93it/s, loss=25.1, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 30:  80%|███▏| 2016/2520 [00:26<00:06, 76.39it/s, loss=24.6, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 31:  80%|███▏| 2016/2520 [00:34<00:08, 58.60it/s, loss=24.2, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 32:  80%|████▊ | 2016/2520 [00:27<00:06, 72.11it/s, loss=24, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 33:  80%|███▏| 2016/2520 [00:28<00:07, 71.71it/s, loss=23.6, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 34:  80%|███▏| 2016/2520 [00:31<00:07, 63.01it/s, loss=23.2, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 35:  80%|███▏| 2016/2520 [00:34<00:08, 58.39it/s, loss=23.1, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Validating:  13%|███▋                       | 68/504 [00:00<00:01, 337.71it/s]


Epoch 36:  80%|███▏| 2016/2520 [00:33<00:08, 60.50it/s, loss=22.6, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 37:  80%|███▏| 2016/2520 [00:29<00:07, 67.26it/s, loss=22.5, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 38:  80%|███▏| 2016/2520 [00:28<00:07, 71.32it/s, loss=22.6, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 39:  80%|███▏| 2016/2520 [00:29<00:07, 67.99it/s, loss=22.1, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 40:  80%|███▏| 2016/2520 [00:26<00:06, 74.83it/s, loss=20.9, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 41:  80%|███▏| 2016/2520 [00:30<00:07, 66.21it/s, loss=20.6, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 42:  80%|████▊ | 2016/2520 [00:27<00:06, 74.29it/s, loss=20, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Validating:  19%|█████▏                     | 96/504 [00:00<00:01, 309.09it/s]


Epoch 43:  80%|████▊ | 2016/2520 [00:26<00:06, 76.22it/s, loss=20, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 44:  80%|███▏| 2016/2520 [00:28<00:07, 70.82it/s, loss=19.9, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 45:  80%|███▏| 2016/2520 [00:27<00:06, 73.13it/s, loss=19.8, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 46:  80%|███▏| 2016/2520 [00:32<00:08, 62.92it/s, loss=19.8, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 47:  80%|███▏| 2016/2520 [00:30<00:07, 65.89it/s, loss=19.7, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 48:  80%|███▏| 2016/2520 [00:30<00:07, 65.65it/s, loss=19.3, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Epoch 49:  80%|███▏| 2016/2520 [00:30<00:07, 66.07it/s, loss=19.3, v_num=4_38]
Validating: 0it [00:00, ?it/s]
Validating:  31%|███████▉                  | 155/504 [00:00<00:00, 368.51it/s]


Epoch 49: 100%|████| 2520/2520 [00:32<00:00, 77.44it/s, loss=19.3, v_num=4_38]


In [15]:
run_experiment(dfp, "dfp_small")
run_experiment(dfp_r, "dfp_r_small")
run_experiment(dfh_r, "dfh_r_small")
run_experiment(dfp_phi, "dfp_phi_small")
run_experiment(dfh_phi, "dfh_phi_small")

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type             | Params
------------------------------------------
0 | enc  | VeloEncoderSmall | 82.4 K
1 | dec  | VeloDecoderSmall | 84.4 K
------------------------------------------
166 K     Trainable params
0         Non-trainable params
166 K     Total params


Epoch 3:  80%|▊| 2016/2520 [00:32<00:08, 62.36it/s, loss=8.46e+04, v_num=8_35]
Validating: 0it [00:00, ?it/s]
Epoch 7:  80%|▊| 2016/2520 [41:14<10:18,  1.23s/it, loss=8.46e+04, v_num=8_35]


Epoch 7: 100%|█| 2520/2520 [41:16<00:00,  1.02it/s, loss=8.46e+04, v_num=8_35]
Epoch 8:  80%|▊| 2016/2520 [8:33:50<2:08:27, 15.29s/it, loss=8.46e+04, v_num=8
Epoch 8:  81%|▊| 2038/2520 [8:33:50<2:01:31, 15.13s/it, loss=8.46e+04, v_num=8
Epoch 8:  82%|▊| 2075/2520 [8:33:50<1:50:11, 14.86s/it, loss=8.46e+04, v_num=8
Epoch 8:  84%|▊| 2112/2520 [8:33:51<1:39:15, 14.60s/it, loss=8.46e+04, v_num=8
Validating:  21%|█████▍                    | 105/504 [00:00<00:01, 336.61it/s]
Epoch 8:  85%|▊| 2149/2520 [8:33:51<1:28:42, 14.35s/it, loss=8.46e+04, v_num=8
Epoch 8:  87%|▊| 2186/2520 [8:33:51<1:18:30, 14.10s/it, loss=8.46e+04, v_num=8
Epoch 8:  88%|▉| 2223/2520 [8:33:51<1:08:39, 13.87s/it, loss=8.46e+04, v_num=8
Epoch 8:  90%|▉| 2260/2520 [8:33:51<59:07, 13.64s/it, loss=8.46e+04, v_num=8_3
Validating:  50%|████████████▉             | 251/504 [00:00<00:00, 282.30it/s]
Epoch 8:  91%|▉| 2297/2520 [8:33:51<49:53, 13.42s/it, loss=8.46e+04, v_num=8_3
Epoch 8:  93%|▉| 2334/2520 [8:33:51<40:57, 13.21s/it

Validating:  48%|████████████▌             | 243/504 [00:00<00:01, 248.28it/s]
Epoch 15:  91%|▉| 2296/2520 [00:30<00:02, 75.30it/s, loss=8.46e+04, v_num=8_35
Validating:  61%|███████████████▊          | 307/504 [00:01<00:00, 276.10it/s]
Epoch 15:  94%|▉| 2378/2520 [00:30<00:01, 77.35it/s, loss=8.46e+04, v_num=8_35
Validating:  76%|███████████████████▊      | 383/504 [00:01<00:00, 317.53it/s]
Epoch 15:  98%|▉| 2460/2520 [00:30<00:00, 79.55it/s, loss=8.46e+04, v_num=8_35
Epoch 15: 100%|█| 2520/2520 [00:31<00:00, 81.08it/s, loss=8.46e+04, v_num=8_35
Epoch 16:  80%|▊| 2016/2520 [00:46<00:11, 43.72it/s, loss=8.46e+04, v_num=8_35
Epoch 16:  81%|▊| 2050/2520 [00:46<00:10, 44.38it/s, loss=8.46e+04, v_num=8_35
Validating:   9%|██▍                        | 45/504 [00:00<00:01, 442.34it/s]
Epoch 16:  85%|▊| 2132/2520 [00:46<00:08, 45.95it/s, loss=8.46e+04, v_num=8_35
Validating:  25%|██████▍                   | 125/504 [00:00<00:00, 416.31it/s]
Epoch 16:  88%|▉| 2214/2520 [00:46<00:06, 47.50it/s,

Validating:  15%|███▉                       | 74/504 [00:00<00:01, 341.29it/s]
Epoch 22:  85%|▊| 2132/2520 [00:23<00:04, 91.79it/s, loss=8.46e+04, v_num=8_35
Validating:  31%|███████▉                  | 155/504 [00:00<00:00, 367.68it/s]
Epoch 22:  88%|▉| 2214/2520 [00:23<00:03, 94.47it/s, loss=8.46e+04, v_num=8_35
Validating:  46%|████████████              | 233/504 [00:00<00:00, 374.05it/s]
Epoch 22:  91%|▉| 2296/2520 [00:23<00:02, 97.07it/s, loss=8.46e+04, v_num=8_35
Validating:  62%|████████████████▏         | 313/504 [00:00<00:00, 383.75it/s]
Epoch 22:  94%|▉| 2378/2520 [00:23<00:01, 99.75it/s, loss=8.46e+04, v_num=8_35
Epoch 22:  98%|▉| 2460/2520 [00:24<00:00, 102.44it/s, loss=8.46e+04, v_num=8_3
Epoch 22: 100%|█| 2520/2520 [00:24<00:00, 104.39it/s, loss=8.46e+04, v_num=8_3
Epoch 23:  80%|▊| 2016/2520 [00:22<00:05, 87.69it/s, loss=8.46e+04, v_num=8_35
Epoch 23:  81%|▊| 2050/2520 [00:23<00:05, 88.87it/s, loss=8.46e+04, v_num=8_35
Validating:  10%|██▌                        | 48/504

...

Epoch 35:  90%|▉| 1136/1260 [00:10<00:01, 108.48it/s, loss=5.98e+04, v_num=6_3
Validating:  51%|█████████████▏            | 128/252 [00:00<00:00, 654.29it/s]
Epoch 35:  96%|▉| 1207/1260 [00:10<00:00, 113.86it/s, loss=5.98e+04, v_num=6_3
Epoch 35: 100%|█| 1260/1260 [00:10<00:00, 117.66it/s, loss=5.98e+04, v_num=6_3
Epoch 36:  80%|▊| 1008/1260 [00:10<00:02, 99.84it/s, loss=5.96e+04, v_num=6_34
Epoch 36:  85%|▊| 1065/1260 [00:10<00:01, 104.55it/s, loss=5.96e+04, v_num=6_3
Epoch 36:  90%|▉| 1136/1260 [00:10<00:01, 110.34it/s, loss=5.96e+04, v_num=6_3
Validating:  52%|█████████████▍            | 130/252 [00:00<00:00, 648.39it/s]
Epoch 36: 100%|█| 1260/1260 [00:10<00:00, 119.94it/s, loss=5.96e+04, v_num=6_3
Epoch 37:  80%|▊| 1008/1260 [00:10<00:02, 98.20it/s, loss=5.94e+04, v_num=6_34
Validating: 0it [00:00, ?it/s]
Epoch 37:  85%|▊| 1065/1260 [00:10<00:01, 102.38it/s, loss=5.94e+04, v_num=6_3
Epoch 37:  90%|▉| 1136/1260 [00:10<00:01, 108.07it/s, loss=5.94e+04, v_num=6_3
Epoch 37:  96%|▉| 120

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type             | Params
------------------------------------------
0 | enc  | VeloEncoderSmall | 82.4 K
1 | dec  | VeloDecoderSmall | 84.4 K
------------------------------------------
166 K     Trainable params
0         Non-trainable params
166 K     Total params


Epoch 6:  80%|████ | 1008/1260 [00:10<00:02, 98.76it/s, loss=71.6, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 9:  80%|████ | 1008/1260 [00:10<00:02, 99.44it/s, loss=65.9, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 12:  80%|███▏| 1008/1260 [00:10<00:02, 91.69it/s, loss=62.2, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 13:  80%|███▏| 1008/1260 [00:10<00:02, 97.76it/s, loss=61.4, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 15:  80%|███▏| 1008/1260 [00:11<00:02, 88.73it/s, loss=58.4, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 16:  80%|███▏| 1008/1260 [00:10<00:02, 93.18it/s, loss=54.7, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 17:  80%|███▏| 1008/1260 [00:10<00:02, 94.22it/s, loss=51.6, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 17:  96%|██▊| 1207/1260 [00:11<00:00, 109.40it/s, loss=51.6, v_num=6_04]


Epoch 18:  80%|███▏| 1008/1260 [00:11<00:02, 90.68it/s, loss=50.6, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 21:  80%|███▏| 1008/1260 [00:10<00:02, 98.65it/s, loss=48.5, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 23:  80%|██▍| 1008/1260 [00:10<00:02, 100.12it/s, loss=46.8, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 24:  80%|████ | 1008/1260 [00:10<00:02, 100.48it/s, loss=46, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 25:  80%|███▏| 1008/1260 [00:10<00:02, 97.62it/s, loss=44.7, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 26:  80%|███▏| 1008/1260 [00:10<00:02, 96.61it/s, loss=43.5, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 27:  80%|██▍| 1008/1260 [00:09<00:02, 103.79it/s, loss=42.4, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 29:  80%|██▍| 1008/1260 [00:09<00:02, 101.96it/s, loss=40.3, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 31:  80%|███▏| 1008/1260 [00:10<00:02, 97.93it/s, loss=38.3, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 33: 

Epoch 37:  80%|███▏| 1008/1260 [00:10<00:02, 92.22it/s, loss=34.8, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 42:  80%|██▍| 1008/1260 [00:10<00:02, 100.02it/s, loss=33.7, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 43:  80%|██▍| 1008/1260 [00:09<00:02, 104.71it/s, loss=33.4, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 44:  80%|███▏| 1008/1260 [00:11<00:02, 89.23it/s, loss=32.9, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 49:  80%|████▊ | 1008/1260 [00:10<00:02, 99.50it/s, loss=32, v_num=6_04]
Validating: 0it [00:00, ?it/s]
Epoch 49: 100%|█████| 1260/1260 [00:10<00:00, 117.45it/s, loss=32, v_num=6_04]


In [17]:
%tensorboard --logdir lightning_logs --host 0.0.0.0


Reusing TensorBoard on port 6006 (pid 627012), started 3:24:48 ago. (Use '!kill 627012' to kill it.)